In [ ]:
# step1.프로젝트에 필요한 패키지 불러오기
from  selenium  import  webdriver
from  selenium.webdriver.common.keys  import  Keys
from  selenium.webdriver.common.by  import  By
import  time
import  pandas  as  pd
import  math
from selenium.webdriver.support.ui import Select
import datetime


# 크롬드라이버 실행 및 잡플래닛 로그인 함수
def  go_to_job(driver):
    driver.get("https://www.jobplanet.co.kr/users/sign_in?_nav=gb")
    time.sleep(1)

    driver.find_element(By.CSS_SELECTOR, 'body > div.body_wrap > div.globalHeaderWrap > header > a').click()
    time.sleep(1)

    driver.find_element(By.CSS_SELECTOR, 'body > div.body_wrap.group\/layout.min-w-\[1060px\].bg-white > div > div.relative.z-\[55\] > header > div.absolute.left-\[164px\].top-0.flex.h-\[100\%\] > a:nth-child(2)').click()
    time.sleep(1)
    
    dropdown_element = driver.find_element(By.CSS_SELECTOR, "#location")
    select = Select(dropdown_element)
    select.select_by_index(1)
    time.sleep(1)


# 원하는 회사의 리뷰 페이지까지 이동 함수
def doScrollDown(Seconds,driver):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=Seconds)
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1)
        if datetime.datetime.now() > end:
            break          

# 하이퍼링크 가져오기
def  get_href(driver):
    div_ls = driver.find_elements(By.CSS_SELECTOR,'body > div.body_wrap.group\/layout.min-w-\[1060px\].bg-white.is-fixed-header > main > div.sc-96d1181c-0.bdeZOM.contents_wrap.relative.z-10.mx-auto.min-h-\[600px\].w-\[1286px\].overflow-x-visible.bg-white.is-banner > div > div.mx-auto.min-h-\[800px\] > div.grid.gap-\[56px_36px\].mt-\[16px\].w-full > div')
    href_ls = []
    for i in div_ls:
        a_tag = i.find_element(By.CSS_SELECTOR, 'a')
        # href 속성 값 가져오기
        href_value = a_tag.get_attribute('href')
        href_ls.append(href_value)
    return href_ls





def  main_1():

    # 크롬 드라이버 실행
    driver  =  webdriver.Chrome()

    # 채용 공고로 가기
    go_to_job(driver)

    # 스크롤
    doScrollDown(2,driver)

    href_ls = get_href(driver)
    print(href_ls)
    # 리뷰 페이지로 이동
    # go_to_review_page(driver,  QUERY)

    # # 리뷰 클롤링
    # total_data  =  scrape_data(driver)

    # # 엑셀 파일로 저장
    # total_data.to_excel(f"잡플래닛 리뷰 총정리_{QUERY}.xlsx",  index=True)

    # # 크롬 드라이버 종료
    # driver.close()



if  __name__  ==  "__main__":
    main_1()

['https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1289676', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1289677', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1289678', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1289679', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1289681', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1289680', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1289692', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1289698', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1289696', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1289617', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1289620', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1289621', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1289618', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1289622', 'https://www.jobplanet.co.kr/job/search?posting

In [ ]:
# def  get_info(driver,href_ls):
#     for href in href_ls:
#         href_selenium = driver.get(href)


def  go_to_review_page(driver, query):

    # 검색창에 회사명 입력
    search_query  =  driver.find_element(By.ID, "search_bar_search_query")
    search_query.send_keys(query)
    search_query.send_keys(Keys.RETURN)
    time.sleep(3)

    # 회사명 클릭
    driver.find_element(
    By.CLASS_NAME, "line-clamp-2.text-h9.text-gray-800").click()
    time.sleep(2)

    # 새로운 창에 출력되므로 최신창으로 변환
    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(5)

    # 리뷰 페이지 클릭
    try:
        driver.find_element(By.CLASS_NAME, "viewReviews").click()
    except:
        pass

    time.sleep(3)



# step5. 별점 변환 함수
def  parse_star_rating(style_attribute):
    if  len(style_attribute)  ==  11:
        rating_value  =  int(style_attribute[7:9])
        return  f"{rating_value  //  20}점"
    else:
        return  "5점"


# step6.데이터 크롤링 함수 (직무/근속여부/일시/요약/평점/장점/단점/경영진에게 바라는 점)
def  scrape_data(driver):

    list_div  =  []
    list_cur  =  []
    list_date  =  []
    list_stars  =  []
    list_summery  =  []
    list_merit  =  []
    list_disadvantages  =  []
    list_opinions  =  []

    # 크롤링 할 리뷰 갯수 파악
    review_count  =  driver.find_element(By.ID, "viewReviewsTitle")
    review_count  =  review_count.find_element(By.TAG_NAME, "span").text

    # 크롤링 할 페이지수 파악
    page  =  math.ceil(int(review_count) /  5)

    for  _  in  range(page):

        review_list  =  driver.find_element(By.ID, "viewReviewsList")
        review_box  =  review_list.find_elements(By.TAG_NAME, "section")

        # 페이지당 최대 5개의 리뷰 박스 존재
        for  i  in  review_box:

            user_info_area  =  i.find_elements(By.TAG_NAME, "div")[0]
            user_info  =  user_info_area.text.split("|")

            # 직무
            division  =  user_info[0]
            list_div.append(division)

            # 재직여부
            current  =  user_info[1]
            list_cur.append(current)

            # 날짜
            try:
                date  =  user_info[3]
                list_date.append(date)

            except: # 날짜 없는 경우 예외처리
                date  =  "날짜 없음"
                list_date.append(date)

            # 리뷰 요약
            try:
                summary  =  i.find_element(By.TAG_NAME, "h2")
                list_summery.append(summary.text)

            except: # 신고로 인해 리뷰 요약 없는 경우 예외처리
                # summery_ban = i.find_element(By.CLASS_NAME, "cont_discontinu.discontinu_category")
                summery_ban  =  "신고로 인해 리뷰 요약 없음"
                list_summery.append(summery_ban)
                list_merit.append(summery_ban)
                list_disadvantages.append(summery_ban)
                list_opinions.append(summery_ban)

            # 장점, 단점, 경영진에게 바라는 점
            try:
                review_info_area  =  i.find_elements(By.TAG_NAME, "dl")[-1]
                contents  =  review_info_area.find_elements(By.TAG_NAME, "dd")

                merit  =  contents[0].text
                list_merit.append(merit)

                disadvantage  =  contents[1].text
                list_disadvantages.append(disadvantage)

                opinion  =  contents[2].text
                list_opinions.append(opinion)

            except:
                pass

            try:
                star_info  =  i.find_elements(By.TAG_NAME, "span")

                has_star  =  False  # 별점이 있는지 확인하기 위한 플래그

                for  star  in  star_info:

                    if  star.get_attribute("style") !=  "": # 빈 값 제외

                        has_star  =  True

                        list_stars.append(
                        parse_star_rating(star.get_attribute("style"))
                        )

                if  not  has_star: # 별점이 없는 경우
                    list_stars.append("별점 없음")

            except  Exception  as  e:
                print(f"별점 없음 - 예외 발생: {str(e)}")
                list_stars.append("별점 없음")

        try:
            driver.find_element(By.XPATH,'//*[@class="btn_pgnext inline-block h-[38px] w-[37px] bg-[url(https://common.jobplanet.co.kr/images/common/global_spt.png)] bg-[-96px_0] bg-no-repeat hover:bg-[-96px_-48px]"]',).click()

            time.sleep(5)

        except:
            pass



    total_data  =  pd.DataFrame({

    '날짜':  list_date,
    '직무':  list_div,
    '고용 현황':  list_cur,
    '별점':  list_stars,
    '요약':  list_summery,
    '장점':  list_merit,
    '단점':  list_disadvantages,
    '경영진에게 바라는 점':  list_opinions

    })

    return  total_data




In [43]:
import requests

response = requests.get("https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1289676")
response

<Response [403]>